In [5]:
import numpy as np
import pandas as pnd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization

dataset = 1
match dataset:
    case 1:
        test = 'fashion-mnist_test.csv'
        train = 'fashion-mnist_train.csv'
    case 2: 
        test = "mnist_test.csv"
        train = "mnist_train.csv"
testdata = pnd.read_csv(test)
traindata = pnd.read_csv(train)

testdata_pixel = testdata.drop(testdata.columns[0], axis=1).to_numpy()
traindata_pixel = traindata.drop(traindata.columns[0], axis=1).to_numpy()
label_train = traindata[traindata.columns[0]].to_numpy()
label_test = testdata[testdata.columns[0]].to_numpy()

cnn_test = testdata_pixel.reshape(-1,28,28) /255.0
cnn_train = traindata_pixel.reshape(-1,28,28) /255.0
cnn_test = np.expand_dims(cnn_test,axis=3)
cnn_train = np.expand_dims(cnn_train,axis=3)
cnn_label_test = tf.keras.utils.to_categorical(label_test)
cnn_label_train = tf.keras.utils.to_categorical(label_train)

softmax_nodes = 10
class_names = np.array(["T-shirt / Top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"])

summarize_shirt = True
if summarize_shirt == True:
    label_test[label_test==6]=0
    label_train[label_train==6]=0
    cnn_label_test = tf.keras.utils.to_categorical(label_test)
    cnn_label_train = tf.keras.utils.to_categorical(label_train)
    softmax_nodes -= 1
    class_names.pop(6)

In [6]:
simple_model = Sequential([
  Conv2D(16, 3, input_shape=(28, 28, 1), activation="relu",padding="same"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(32, 3, activation="relu",padding="valid"),
  MaxPooling2D(pool_size=2,strides=2),
  Conv2D(64, 3, activation="relu",padding="valid"),
  Flatten(),
  Dense(256,activation="relu"),
  Dense(softmax_nodes, activation='softmax'),
])
simple_model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
simple_model.fit(
  cnn_train,
  cnn_label_train,
  epochs=10,
  validation_data=(cnn_test, cnn_label_test)
)

Epoch 1/10
1875/1875 [==============================] - 24s 12ms/step - loss: 0.4431 - accuracy: 0.8376 - val_loss: 0.3117 - val_accuracy: 0.8799
Epoch 2/10
1875/1875 [==============================] - 28s 15ms/step - loss: 0.2868 - accuracy: 0.8952 - val_loss: 0.2495 - val_accuracy: 0.9069
Epoch 3/10
1875/1875 [==============================] - 54s 29ms/step - loss: 0.2440 - accuracy: 0.9099 - val_loss: 0.2449 - val_accuracy: 0.9087
Epoch 4/10
1875/1875 [==============================] - 40s 21ms/step - loss: 0.2138 - accuracy: 0.9213 - val_loss: 0.2255 - val_accuracy: 0.9165
Epoch 5/10
1875/1875 [==============================] - 46s 25ms/step - loss: 0.1885 - accuracy: 0.9300 - val_loss: 0.2166 - val_accuracy: 0.9193
Epoch 6/10
1052/1875 [===============>..............] - ETA: 21s - loss: 0.1660 - accuracy: 0.9373

In [4]:
import sklearn
import pandas as pd
from sklearn.metrics import confusion_matrix
y_pred = simple_model.predict(cnn_test)
y_pred = np.argmax(y_pred,axis=1)
conf = sklearn.metrics.confusion_matrix(y_pred, label_test)
conf_df = pd.DataFrame(conf, index=class_names, columns=class_names)
conf_df = round(conf_df*100 / conf_df.sum(axis=1),2)
print(conf_df)

NameError: name 'simple_model' is not defined